In [2]:
import pandas as pd
import folium
import numpy as np

In [9]:
df = pd.read_csv('data.csv', delimiter=',')
df

,id,region,municipality,settlement,type,latitude_dd,longitude_dd
0,0,Республика Алтай,Шебалинский район,Каспа,с,5111,8601
1,1,Алтайский край,Смоленский,Молочный,п,5241,8497
2,2,Красноярский край,Казачинский район,Отношка,с,5738,9270
3,3,Республика Тыва,Каа-Хемский кожуун,Кундустуг,с,5157,9518
4,4,Красноярский край,Курагинский район,Щетинкино,с,5453,9344
...,...,...,...,...,...,...,...
6132,6132,Новосибирская область,Сузунский район,Мереть,с,5357,8239
6133,6133,Новосибирская область,Сузунский район,Маюрово,с,5431,8242
6134,6134,Алтайский край,Быстроистокский,Смоленский,п,5221,8466
6135,6135,Новосибирская область,Искитимский район,Малиновка,д,5466,8386


In [10]:
n = len(df)
dist_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        if i == j:
            continue
        coords_i = (df.loc[i, 'latitude_dd'], df.loc[i, 'longitude_dd'])
        coords_j = (df.loc[j, 'latitude_dd'], df.loc[j, 'longitude_dd'])
        dist = np.sqrt(np.sum(np.square(np.array(coords_i) - np.array(coords_j))))
        dist_matrix[i, j] = dist
np.savetxt('dist_matrix.csv', dist_matrix, delimiter='|', fmt="%.4f")

In [11]:
dist_matrix = np.loadtxt('dist_matrix.csv', delimiter='|')
dist_submatrix = dist_matrix[:100, :100]
np.savetxt('dist_submatrix.csv', dist_submatrix, delimiter='|', fmt="%.3f")

In [12]:
dist_matrix

array([[  0.    , 166.4812, 916.8915, ..., 174.1407, 415.0301, 678.1983],
       [166.4812,   0.    , 918.9875, ...,  36.8917, 250.8904, 682.8214],
       [916.8915, 918.9875,   0.    , ..., 955.8792, 924.9   , 239.0586],
       ...,
       [174.1407,  36.8917, 955.8792, ...,   0.    , 257.7305, 719.6721],
       [415.0301, 250.8904, 924.9   , ..., 257.7305,   0.    , 711.6832],
       [678.1983, 682.8214, 239.0586, ..., 719.6721, 711.6832,   0.    ]])

In [13]:
matrix = dist_matrix
len(matrix)

6137

In [40]:
def greedy(matrix, start_vertex=0, way_fp="way.csv"):
    n = matrix.shape[0]
    visited = np.zeros(n, dtype=bool)
    start_vertex_v = start_vertex
    edges = np.zeros(n + 1, dtype=int)

    for i in range(n):
        visited[start_vertex_v] = True
        min_distance = np.inf
        min_vertex = -1
        for j in range(n):
            if j != start_vertex_v and not visited[j]:
                distance = matrix[start_vertex_v, j]
                if distance < min_distance:
                    min_distance = distance
                    min_vertex = j
        edges[i] = start_vertex_v
        start_vertex_v = min_vertex
    edges[n] = edges[0]
    edges = np.array(edges)
    np.savetxt(way_fp, edges, fmt="%d")
    print("Путь сохранен в", way_fp)
    return edges

def find_len(matrix, way):
    return sum([matrix[way[i]][way[i + 1]] for i in range(0, len(way) - 1)]) / 100

def save_to_kaggle(matrix, way):
    df = pd.DataFrame(columns=["Id", "Predicted"])
    for i in range(len(way) - 1):
        id1, id2 = way[i], way[i + 1]
        dist = matrix[id1][id2] / 100
        df.loc[i] = [id1, dist]

    df["Id"] = df["Id"].astype(int)
    df["Predicted"] = df["Predicted"].astype(float)
    df.to_csv("to_kaggle.csv", index=False, sep=",")


In [42]:
way = greedy(dist_matrix, start_vertex=3753, way_fp="way.csv")
print(find_len(dist_matrix, way))

Путь сохранен в way.csv
1074.3789600000018


In [43]:
loaded_way = np.loadtxt("best_way.csv", dtype=int)
print(f"После улучшения: {find_len(dist_matrix, loaded_way)}")

После улучшения: 892.1282730000011


In [ ]:
save_to_kaggle(dist_matrix, loaded_way)